In [1]:
import cadquery as cq

# Create a sloped box: 50x50x10, then rotate 20° about X to tilt the top
sloped_box = (
    cq.Workplane("XY")
    .box(50, 50, 10)
    .rotate((0, 0, 0), (1, 0, 0), 20)  # rotate around X axis
)

# Export
cq.exporters.export(sloped_box, 'sloped_box.step')

In [2]:
import cadquery as cq

# Create a box at the origin (centered by default)
box = cq.Workplane("XY").box(50, 50, 10)  # width, depth, height

# Export to a STEP file
cq.exporters.export(box, 'box_50x50.step')


In [3]:
model = cq.importers.importStep('box_50x50.step')

In [2]:
import cadquery as cq
from tqdm import tqdm


# Load your sloped STEP file
model = cq.importers.importStep("face v1.step")

# Get bounding box
bbox = model.val().BoundingBox()
xmin, xmax = bbox.xmin, bbox.xmax
ymin, ymax = bbox.ymin, bbox.ymax
zmin = bbox.zmin  # This is the minimum Z value (bottom of the bounding box)

# Parameters
COEFFICIENT = 1.0
spacing = 4 * COEFFICIENT
cone_radius = 4 * COEFFICIENT # FYI: This cannot be bigger than spacing otherwise we get a kernel crash...
assert cone_radius <= spacing, "cone_radius must be less than or equal to spacing to avoid kernel crash"
cone_depth = 4.0 * COEFFICIENT

RAISED_DEPTH = 0  # cone_depth / 2
connecting_cylinder_radius = 0.75 * COEFFICIENT
connecting_cylinder_depth = cone_depth  # 0.5 * cone_depth
sphere_radius = 1.5 * COEFFICIENT
sphere_depth = connecting_cylinder_depth * 1.5  # (cone_depth + (sphere_radius / 4))

# Build divot geometry on the actual 3D surface
divots = cq.Workplane("XY")

for x_idx, x in tqdm(enumerate(range(int(xmin), int(xmax) + 2, int(spacing)))):
    # TODO: take me out...
    # if x_idx <= 7:
    #     continue
    for y in range(int(ymin), int(ymax) + 2, int(spacing)):
        # Create a ray (small box pointing downward) that starts below the object's surface
        ray = cq.Solid.makeBox(0.1, 0.1, 100).translate((x, y, zmin - 50))

        # Perform intersection to find where the ray meets the surface
        intersection = model.intersect(ray)

        try:
            # Get the highest Z value in the intersected shape
            z_top = intersection.val().BoundingBox().zmax
            top_cylinder = (
                cq.Workplane("XY")
                .cylinder(cone_depth * 4, cone_radius)
                .translate((0, 0, cone_depth * 2 + RAISED_DEPTH))
            )  # Extend the cylinder above the surface

            cone = (
                cq.Workplane("XY")
                .circle(cone_radius)
                .workplane(offset=-cone_depth)
                .center(0, 0)  # make sure it's aligned
                .circle(
                    0.0001
                )  # small non-zero radius instead of 0 to avoid geometry errors
                .loft()
                .translate((0, 0, RAISED_DEPTH))
            )

            connecting_cylinder = (
                cq.Workplane("XY")
                .cylinder(connecting_cylinder_depth, connecting_cylinder_radius)
                .translate((0, 0, RAISED_DEPTH - connecting_cylinder_depth))
            )

            # Create the sphere at the tip
            sphere = (
                cq.Workplane("XY")
                .sphere(sphere_radius)
                .translate((0, 0, RAISED_DEPTH - sphere_depth))
            )

            # Combine them
            divot = cone.union(sphere).union(top_cylinder).union(connecting_cylinder)

            # Translate the divot to the correct position, starting from the object's surface
            divot = divot.translate((x, y, z_top))

            # Add to combined divots
            divots = divots.union(divot)

            # if debug:
            #     # Subtract divots from original model
            #     modified = model.cut(divots)

            #     # Export result
            #     cq.exporters.export(modified, f"divotted_face_v1_{x}{y}.step")
        except Exception as e:
            print("skipping divot at ({}, {}): {}".format(x, y, e))
            continue

# Subtract divots from original model
modified = model.cut(divots)

# Export result
cq.exporters.export(modified, "divotted_face_v1.step")

0it [00:00, ?it/s]

skipping divot at (-72, -43): Bnd_Box is void
skipping divot at (-72, -39): Bnd_Box is void
skipping divot at (-72, -35): Bnd_Box is void
skipping divot at (-72, -31): Bnd_Box is void
skipping divot at (-72, -27): Bnd_Box is void
skipping divot at (-72, -23): Bnd_Box is void
skipping divot at (-72, -19): Bnd_Box is void
skipping divot at (-72, -15): Bnd_Box is void
skipping divot at (-72, -11): Bnd_Box is void
skipping divot at (-72, -7): Bnd_Box is void
skipping divot at (-72, -3): Bnd_Box is void
skipping divot at (-72, 17): Bnd_Box is void
skipping divot at (-72, 21): Bnd_Box is void
skipping divot at (-72, 25): Bnd_Box is void
skipping divot at (-72, 29): Bnd_Box is void
skipping divot at (-72, 33): Bnd_Box is void
skipping divot at (-72, 37): Bnd_Box is void
skipping divot at (-72, 41): Bnd_Box is void
skipping divot at (-72, 45): Bnd_Box is void
skipping divot at (-72, 49): Bnd_Box is void
skipping divot at (-72, 53): Bnd_Box is void


1it [02:43, 163.09s/it]

skipping divot at (-72, 57): Bnd_Box is void
skipping divot at (-68, -43): Bnd_Box is void
skipping divot at (-68, -39): Bnd_Box is void
skipping divot at (-68, -35): Bnd_Box is void
skipping divot at (-68, -31): Bnd_Box is void
skipping divot at (-68, -27): Bnd_Box is void
skipping divot at (-68, -23): Bnd_Box is void
skipping divot at (-68, -19): Bnd_Box is void


1it [04:48, 288.27s/it]


KeyboardInterrupt: 